In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


In [ ]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [ ]:
# LOCAL imports
import rts
import rts.pipeline
import rts.utils
import rts.io.media
import rts.features.audio
import rts.features.text

LOG = rts.utils.get_logger()

In [ ]:
# vidx = rts.metadata.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.metadata.read_all_metadata(METADATA, vidx)
# rts.metadata.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

# df = rts.metadata.load_metadata_hdf5(METADATA, 'rts_metadata')
# sample_df = rts.metadata.get_one_percent_sample(df)
# # rts.metadata.export_metadata_stats(sample_df, Path(OUTDIR) / 'sample')
# # rts.metadata.export_metadata_stats(df, OUTDIR)
# sample_df.head()

medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)
# clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

df = rts.metadata.create_clips_df(LOCAL_VIDEOS)

In [ ]:
# Database fulltext

In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [69]:
medias = rts.metadata.load_all_media_info(LOCAL_VIDEOS)

In [71]:
clips = rts.metadata.load_all_clips(LOCAL_VIDEOS)

In [73]:
df = rts.metadata.create_clips_df(LOCAL_VIDEOS)

In [83]:
atlases = rts.metadata.create_clip_texture_atlases(df, LOCAL_RTS_DATA, tile_size=128, format='jpg')

In [ ]:
import PIL
import os
from PIL import ImageOps, Image

In [15]:
test_im_path = os.path.join(OUTDIR, 'test_image.jpg')
im = Image.open(test_im_path)
# rts.io.media.save_image_pyramid(im, Path(OUTDIR) / 'test_image_pyramid', 'test_image', True)

In [ ]:
video_path = os.path.join(LOCAL_VIDEOS, '0', '0', '0', 'ZB001000', 'ZB001000.mp4')
# rts.io.media.get_media_info(video_path)

# scenes = rts.io.media.detect_scenes(video_path)
# scenes = rts.io.media.filter_scenes(scenes, 5)
# clips = rts.io.media.save_scenes_images(scenes, video_path, OUTDIR)
# rts.pipeline.save_scenes(clips, OUTDIR)

In [ ]:
transcript = rts.pipeline.transcribe_media(
    OUTDIR,
    None, 
    force=False
)

t = rts.features.text.find_locations(transcript)

In [ ]:
clips = rts.pipeline.extract_location_clips_from_transcript(transcript,
    25,
    OUTDIR,
    video_path,
    min_seconds=3,
    extend_duration=6,
    num_images=3,
    force=True
)

In [64]:
i1 = os.path.join(OUTDIR, 'test128px.jpg')
i2 = os.path.join(OUTDIR, 'test128px2.jpg')

ori1 = os.path.join(OUTDIR, 'testori1.jpg')
ori2 = os.path.join(OUTDIR, 'testori2.jpg')
imgs = [i1, i2, ori1, ori2]
# imgs = [i1, i2, i1, i2]

# rts.io.media.create_image_grid(
#     imgs,
#     Path(OUTDIR) / 'grid.jpg',
#     grid_size=(2, 1),
#     grid_border=0,
# )

In [50]:
rts.io.media.create_atlas_texture(
    imgs,
    Path(OUTDIR) / 'atlas' / 'atlas.png',
    max_width=1024,
    max_tile_size=512,
    square=True
)

{'images': ['data/test128px.jpg',
  'data/test128px2.jpg',
  'data/testori1.jpg',
  'data/testori2.jpg'],
 'image_count': 4,
 'tile_size': (128, 128),
 'atlas_size': (1024, 1024),
 'rows': 8,
 'cols': 8}

In [66]:
rts.io.media.create_square_atlases(
    imgs,
    Path(OUTDIR) / 'atlas',
    width=256,
    max_tile_size=128,
    bg_color=(255, 255, 255, 255)
)

{'0': {'images': ['data/test128px.jpg',
   'data/test128px2.jpg',
   'data/testori1.jpg',
   'data/testori2.jpg'],
  'image_count': 4,
  'tile_size': (128, 128),
  'atlas_size': (256, 256),
  'rows': 2,
  'cols': 2,
  'path': 'data/atlas/atlas000.png'}}

In [ ]:
res = rts.features.text.enrich_scenes_with_transcript(clips.copy(), t)
# res

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
# remuxed = rts.pipeline.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
# print(remuxed)

rts.pipeline.process_media(input_video_folder, LOCAL_VIDEOS, 
    compute_transcript=True, force_scene=False, force_trans=False)

In [ ]:
%ls {remux_folder}

In [ ]:
fts.groupby('mediaId').count().sort_values('location', ascending=False)

In [ ]:
fts

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8080") # or another location where your Weaviate instance is running

client.schema.get() # get the full schema as example

In [ ]:
swiss_cities = rts.features.text.get_swiss_cities()

In [ ]:
swiss_cities['Genève']

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

In [ ]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    # print(stream.codec_context.height)
    for i, frame in enumerate(container.decode(stream)):
        if i % 4 != 0:
            continue


    #     print(frame)
    #     print(frame.time, frame.time_base)
        # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
        frame.to_image().save(
            os.path.join(out_path, f"f{frame.pts:04d}.jpg"),
            quality=80,
        )

In [ ]:
thumbs_dir = os.path.join(OUTDIR, 'thumbs')
outfast = './notebook/fastdup'
fastdup_data = './notebook/out'

In [ ]:
import fastdup

In [ ]:

fastdup.run(input_dir=thumbs_dir, work_dir=fastdup_data, nearest_neighbors_k=8, turi_param='ccthreshold=0.90')    #main running function.
# fastdup.create_duplicates_gallery('out/similarity.csv', save_path=outfast)     #create a visual gallery of found duplicates
# fastdup.create_outliers_gallery('out/outliers.csv',   save_path=outfast)       #create a visual gallery of anomalies
# fastdup.create_components_gallery(fastdup_data, save_path=outfast)                    #create visualiaiton of connected components
# fastdup.create_stats_gallery('out', save_path=outfast, metric='blur')          #create visualization of images stastics (for example blur)
# fastdup.create_similarity_gallery('out', save_path=outfast,get_label_func=lambda x: x.split('/')[-2])     #create visualization of top_k similar images assuming data have labels which are in the folder name
# fastdup.create_aspect_ratio_gallery('out', save_path=outfast)

In [ ]:
config = fastdup.load_config(fastdup_data)
df = fastdup.galleries.do_find_top_components(work_dir=fastdup_data, input_dir=config['input_dir'], kwargs={})